# Example: Custom `MoveStrategy`: RepEx-Shoot-Repex

One of the powerful features in OpenPathSampling is that it is very easy to develop new Monte Carlo movers for path space. This example shows how easy it is to try out a new type of move. The particular move we use here can be easily described as a simple combination of existing moves, so we don't even need to define a new `PathMover` subclass. We just define a custom `MoveStrategy` that creates the desired `PathMover`, and use that directly.

The idea implemented here is pretty simple. Our standard path movers treat shooting and replica exchange separately, and each move is a single shooting (one ensemble) or a single replica exchange (swap one pair). But maybe you could get better replica exchange behavior by trying all the replica exchange moves, and then trying all the shooting moves. Note that, to satisfy detailed balance, you really have to do all the replica exchange moves, then all the shooting moves, then all the replica exchange moves in the reverse order from before. To measure how this affects travel in replica space, we'll use the replica round trip time (normalized to the total number of shooting moves per ensemble).

In [1]:
import openpathsampling as paths
import numpy as np

# Set up the simulation

## Set up the engine

In [2]:
import openpathsampling.engines.toy as toys
pes = (toys.OuterWalls([1.0,1.0], [0.0,0.0]) + 
       toys.Gaussian(-0.7, [12.0, 0.5], [-0.5, 0.0]) +
       toys.Gaussian(-0.7, [12.0, 0.5], [0.5, 0.0]))

topology = toys.Topology(n_spatial=2, masses=[1.0, 1.0], pes=pes)

engine = toys.Engine(options={'integ': toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5),
                              'n_frames_max': 5000,
                              'n_steps_per_frame': 10},
                     topology=topology)

template = toys.Snapshot(coordinates=np.array([[0.0, 0.0]]),
                         velocities=np.array([[0.0, 0.0]]),
                         engine=engine)

## Set up CV and volumes (states, interfaces)

In [3]:
# states are volumes in a CV space: define the CV
def xval(snapshot):
    return snapshot.xyz[0][0]

cv = paths.FunctionCV("xval", xval)

stateA = paths.CVDefinedVolume(cv, float("-inf"), -0.5).named("A")
stateB = paths.CVDefinedVolume(cv, 0.5, float("inf")).named("B")
interfaces_AB = paths.VolumeInterfaceSet(cv, float("-inf"), [-0.5, -0.4, -0.3, -0.2, -0.1, 0.0])

## Set up network

In [4]:
network = paths.MISTISNetwork([(stateA, interfaces_AB, stateB)])

## Define a custom strategy

This is the main point of this example: everything else is covered in other examples as well. Here we create a custom `MoveStrategy`

In [5]:
import openpathsampling.analysis.move_strategy as strategies # TODO: handle this better
# example: custom subclass of `MoveStrategy`
class RepExShootRepExStrategy(strategies.MoveStrategy):
    _level = strategies.levels.GROUP
    # we define an init function mainly to set defaults for `replace` and `group`
    def __init__(self, ensembles=None, group="repex_shoot_repex", replace=True, network=None):
        super(RepExShootRepExStrategy, self).__init__(
            ensembles=ensembles, group=group, replace=replace
        )
            
    def make_movers(self, scheme):
        # if we replace, we remove these groups from the scheme.movers dictionary
        if self.replace:
            repex_movers = scheme.movers.pop('repex')
            shoot_movers = scheme.movers.pop('shooting')
        else:
            repex_movers = scheme.movers['repex']
            shoot_movers = scheme.movers['shooting']
        # combine into a list for the SequentialMover
        mover_list = repex_movers + shoot_movers + list(reversed(repex_movers))
        combo_mover = paths.SequentialMover(mover_list)
        return [combo_mover]

## Create two move schemes: Default and Custom

In [6]:
default_scheme = paths.DefaultScheme(network, engine)

In [7]:
custom_scheme = paths.DefaultScheme(network, engine)
custom_scheme.append(RepExShootRepExStrategy())

# Get initial conditions

In [8]:
initial_samples = paths.FullBootstrapping(transition=network.sampling_transitions[0],
                                          snapshot=template,
                                          engine=engine).run()

DONE! Completed Bootstrapping cycle step 110 in ensemble 6/6.


In [9]:
transition = network.sampling_transitions[0]
minus_sample = network.minus_ensembles[0].populate_minus_ensemble(
    partial_traj=initial_samples[transition.ensembles[0]].trajectory,
    minus_replica_id=-1,
    engine=engine
)
initial_samples = initial_samples.apply_samples(minus_sample)

In [10]:
initial_samples.sanity_check()

In [11]:
print "Default Scheme:", default_scheme.initial_conditions_report(initial_samples)
print "Custom Scheme:", custom_scheme.initial_conditions_report(initial_samples)

Default Scheme: No missing ensembles.
No extra ensembles.

Custom Scheme: No missing ensembles.
No extra ensembles.



# Run each of the simulations

In [12]:
n_tries_per_shooting = 200

In [13]:
# take the number of steps from a single ensemble shooting
n_steps = default_scheme.n_steps_for_trials(
    mover=default_scheme.movers['shooting'][0],
    n_attempts=n_tries_per_shooting
)
n_steps = int(n_steps)+1
print n_steps

2341


In [14]:
default_storage = paths.Storage("default_scheme.nc", "w")

In [15]:
default_calc = paths.PathSampling(
    storage=default_storage,
    sample_set=initial_samples,
    move_scheme=default_scheme
)

In [16]:
default_calc.run(n_steps)

Working on Monte Carlo cycle number 2341.
DONE! Completed 2341 Monte Carlo cycles.


In [17]:
# in repex_shoot_repex, one move shoots all the ensembles
n_steps = custom_scheme.n_steps_for_trials(
    mover=custom_scheme.movers['repex_shoot_repex'],
    n_attempts=n_tries_per_shooting
)
n_steps = int(n_steps)+1
print n_steps

841


In [18]:
custom_storage = paths.Storage("custom_scheme.nc", "w")

In [19]:
custom_calc = paths.PathSampling(
    storage=custom_storage,
    sample_set=initial_samples,
    move_scheme=custom_scheme
)

In [20]:
custom_calc.run(n_steps)

Working on Monte Carlo cycle number 841.
DONE! Completed 841 Monte Carlo cycles.


# Analyze the results

## A few checks that we are making a fair comparison

### The scheme should be as expected

In [21]:
default_scheme.move_summary(default_storage.steps)

repex ran 22.683% (expected 21.37%) of the cycles with acceptance 234/531 (44.07%)
shooting ran 51.217% (expected 51.28%) of the cycles with acceptance 830/1199 (69.22%)
minus ran 1.751% (expected 1.71%) of the cycles with acceptance 41/41 (100.00%)
pathreversal ran 24.349% (expected 25.64%) of the cycles with acceptance 403/570 (70.70%)


In [22]:
custom_scheme.move_summary(custom_storage.steps)

repex_shoot_repex ran 21.284% (expected 23.81%) of the cycles with acceptance 179/179 (100.00%)
minus ran 4.043% (expected 4.76%) of the cycles with acceptance 34/34 (100.00%)
pathreversal ran 74.673% (expected 71.43%) of the cycles with acceptance 509/628 (81.05%)


### The number of snapshots generated by each should be similar

In [23]:
print len(default_storage.snapshots), len(custom_storage.snapshots)

116158 95220


### Check that we have about the same number of shooting moves per ensemble for each scheme

In [24]:
default_scheme.move_summary(default_storage.steps, "shooting")

OneWayShootingMover [TISTransition] 4 ran 8.672% (expected 8.55%) of the cycles with acceptance 131/203 (64.53%)
OneWayShootingMover [TISTransition] 3 ran 9.013% (expected 8.55%) of the cycles with acceptance 142/211 (67.30%)
OneWayShootingMover [TISTransition] 0 ran 8.501% (expected 8.55%) of the cycles with acceptance 169/199 (84.92%)
OneWayShootingMover [TISTransition] 2 ran 9.056% (expected 8.55%) of the cycles with acceptance 153/212 (72.17%)
OneWayShootingMover [TISTransition] 5 ran 8.159% (expected 8.55%) of the cycles with acceptance 101/191 (52.88%)
OneWayShootingMover [TISTransition] 1 ran 7.817% (expected 8.55%) of the cycles with acceptance 134/183 (73.22%)


In [25]:
custom_scheme.move_summary(custom_storage.steps, "repex_shoot_repex")

Sequential ran 21.284% (expected 23.81%) of the cycles with acceptance 179/179 (100.00%)


## Analyze the output to compare the efficiency

### Count the number of round trips done

In [26]:
default_repx_net = paths.ReplicaNetwork(default_scheme, default_storage.steps)

In [27]:
default_trips = default_repx_net.trips(bottom=network.minus_ensembles[0], top=network.all_ensembles[-1])

In [28]:
n_default_round_trips = len(default_trips['round'])
print n_default_round_trips

16


In [29]:
custom_repx_net = paths.ReplicaNetwork(custom_scheme, custom_storage.steps)

In [30]:
custom_trips = custom_repx_net.trips(bottom=network.minus_ensembles[0], top=network.all_ensembles[-1])

In [31]:
n_custom_round_trips = len(custom_trips['round'])
print n_custom_round_trips

14


### Check the replica flow for each scheme

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
# TODO: move some of the pandas stuff into the analysis objects

In [39]:
ensemble_list = network.minus_ensembles + network.all_ensembles
ensemble_id = {e: ensemble_list.index(e) for e in ensemble_list}
default_flow = pd.Series(default_repx_net.flow(bottom=network.minus_ensembles[0], top=network.all_ensembles[-1]))
#default_flow.index = [ensemble_id[e] for e in default_flow.index]
# TODO: this doesn't seem to be right